# Welcome to the Snowflake Cortex AISQL Demonstration Notebook 👋
This notebook provides a practical overview of Snowflake's AI capabilities for extracting insights from unstructured data sources such as documents, images, and more. Using [Snowflake Cortex AISQL](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#label-cortex-llm-ai-function) functions, we will demonstrate how to seamlessly integrate AI-powered analysis within your data workflows.

## What to Expect in This Notebook
- 📁 Set up an internal Snowflake stage and upload sample files
- 🔍 Explore the contents of the stage using SQL queries
- 🤖 Apply Cortex AISQL functions to extract meaningful information from the uploaded files

## Available AISQL Functions (as of August 2025)
The following AISQL functions are currently supported:

| Function | Description |
| ----------- | ----------- |
| [AI_COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/ai_complete) | Generates a completion for a given text string or image using a selected LLM. Use this function for most generative AI tasks. |
| [AI_CLASSIFY](https://docs.snowflake.com/en/sql-reference/functions/ai_classify) | Classifies text or images into user-defined categories. |
| [AI_FILTER](https://docs.snowflake.com/en/sql-reference/functions/ai_filter) | Returns True or False for a given text or image input, allowing you to filter results in SELECT, WHERE, or JOIN ... ON clauses. |
| [AI_AGG](https://docs.snowflake.com/en/sql-reference/functions/ai_agg) | Aggregates a text column and returns insights across multiple rows based on a user-defined prompt. |
| [AI_EMBED](https://docs.snowflake.com/en/sql-reference/functions/ai_embed) |  Generates an embedding vector for a text or image input, which can be used for similarity search, clustering, and classification tasks. |
| [AI_EXTRACT](https://docs.snowflake.com/en/sql-reference/functions/ai_extract) | Extracts information from an input string or file, for example, text, images, and documents. Supports multiple languages. |
| [AI_SUMMARIZE_AGG](https://docs.snowflake.com/en/sql-reference/functions/ai_summarize_agg) | Aggregates a text column and returns a summary across multiple rows. This function isn’t subject to context window limitations. |
| [AI_SIMILARITY](https://docs.snowflake.com/en/sql-reference/functions/ai_summarize_agg) | Calculates the embedding similarity between two inputs. |
| [AI_TRANSCRIBE](https://docs.snowflake.com/en/sql-reference/functions/ai_transcribe) | Transcribes audio files stored in a stage, extracting text, timestamps, and speaker information. |
| [AI_PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/ai_parse_document) | Extracts text (using OCR mode) or text with layout information (using LAYOUT mode) from documents in an internal or external stage. |
| [TRANSLATE](https://docs.snowflake.com/en/sql-reference/functions/translate-snowflake-cortex) | Translates text between supported languages. |
| [AI_SENTIMENT](https://docs.snowflake.com/en/sql-reference/functions/ai_sentiment) | Extracts sentiment scores from text. |
| [EXTRACT_ANSWER](https://docs.snowflake.com/en/sql-reference/functions/ai_sentiment) | Extracts the answer to a question from unstructured data, provided that the relevant data exists. |
| [SUMMARIZE](https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex) | Returns a summary of the text that you’ve specified. |


---
## Additional Information 📚

To deepen your understanding of Snowflake Cortex AISQL capabilities, consider exploring the following key areas:

#### 🔍 Documentation Links

- [Available Models](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#regional-availability)
- [Cost Considerations](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#cost-considerations)
- [Track credit comsuption for AISQL functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#track-credit-consumption-for-aisql-functions)
- [Choosing a model](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#choosing-a-model)

#### 💡 Track AISQL Usage and Cost
Use the following query to monitor credit consumption associated with AISQL function usage:

```sql
SELECT *
  FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY;
```

---

## Snowflake Environment Setup ❄️
Before proceeding, ensure that you are working within the appropriate Snowflake schema and that the required internal stage for file storage is available.

This step is essential to enable file uploads and subsequent processing using Cortex AISQL functions.

---

In [ ]:
-- Setup objects
USE ROLE FRANCISCO_LEYVA_ROLE;
CREATE SCHEMA IF NOT EXISTS FRANCISCO_SCHEMA;

-- Move the appropriate schema
USE SCHEMA FRANCISCO_SCHEMA;
USE WAREHOUSE QUERY_WH;

-- Create stage (if missing)
CREATE OR ALTER STAGE FRANCISCO_STAGE
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
DIRECTORY = (ENABLE = TRUE);

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Exploring the Internal Stage 📂
In this section, we will browse the contents of the internal Snowflake stage to verify that the files have been uploaded successfully. This step ensures that the data is accessible and ready for processing using Cortex AISQL functions.

You may also upload additional files at this point, if needed, to expand the scope of the analysis.

In [ ]:
-- List directory content
SELECT  * FROM DIRECTORY('@FRANCISCO_STAGE');

In [ ]:
-- List files (images) under the "kitchens" path
SELECT *, 
    TRY_TO_FILE(FILE_URL) AS FILE_IMAGE,
    GET_PRESIGNED_URL(@FRANCISCO_STAGE,RELATIVE_PATH) AS PRESIGNED_URL
FROM DIRECTORY('@FRANCISCO_STAGE')
WHERE STARTSWITH(RELATIVE_PATH,'kitchens');

In [ ]:
import pandas as pd
import streamlit as st

# Load Snowflake stage file data
df_stage = sql_files.to_pandas()

st.title("Images Table")

# Build DataFrame with additional columns
df = pd.DataFrame(
    {
        "ID": df_stage["RELATIVE_PATH"],
        "IMAGE": df_stage["PRESIGNED_URL"],
        "URL": df_stage["PRESIGNED_URL"]
    }
)

# Display the DataFrame in Streamlit
st.dataframe(
    df,
    column_config={
        "ID": "ID",
        "IMAGE": st.column_config.ImageColumn("Image", width="medium"),
        "URL": st.column_config.LinkColumn("URL")
    },
    use_container_width=False,
    height=400,
    hide_index=True
)


## Applying Snowflake AI SQL Functions 🤖
In this section, we will leverage Snowflake Cortex AISQL functions to extract meaningful insights from image files stored in the internal stage.

Specifically, we will use the AI_COMPLETE and AI_CLASSIFY functions to:

- Generate and extract relevant information from each image
- Store the results in a temporary table for structured analysis
- Query and explore the extracted insights directly from the table

This approach demonstrates how Snowflake’s native AI capabilities can be seamlessly integrated into your data workflows to analyze unstructured content at scale.

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE KITCHENS_DATA AS
SELECT

    -- Image identifier
    RELATIVE_PATH,

    -- Generate an image description
    CONCAT('This is a picture describing the kitchen: '|| RELATIVE_PATH || ' This is the description: ' ||
        AI_COMPLETE('claude-3-5-sonnet', 'Describe this image: ', TO_FILE('@FRANCISCO_STAGE', RELATIVE_PATH))) AS DESCRIPTION,

    -- Classify this kitchen as modern, industrial or vintage
    -- AI_COMPLETE('claude-3-5-sonnet', 'Classify this image, respond only with Modern or Traditional: ',  TO_FILE('@FRANCISCO_STAGE', RELATIVE_PATH)) AS STYLE,
    AI_CLASSIFY(TO_FILE('@FRANCISCO_STAGE', RELATIVE_PATH), ['modern', 'industrial', 'vintage']):labels AS STYLE,

    -- Generate and extract the kitchen appliances for each image
    AI_COMPLETE('claude-3-5-sonnet', 'Extract the kitchen appliances identified in this image. Return an array only with the identified appliances.',
    TO_FILE('@FRANCISCO_STAGE', RELATIVE_PATH)) AS APPLIANCES
    
FROM 
    {{sql_files}}
WHERE STARTSWITH(RELATIVE_PATH, 'kitchens');

In [ ]:
-- Show kitchens data
SELECT * FROM KITCHENS_DATA;

### Using AI_FILTER to Locate Specific Records 🔎

The AI_FILTER function enables intelligent querying of unstructured data using natural language. In this section, we demonstrate how to apply AI_FILTER to search for specific records from the kitchens table using generative AI.

In [ ]:
-- Find a specific kitchen using AI_FILTER
SELECT *, TRUE AS HAS_LEMONS
FROM KITCHENS_DATA
WHERE AI_FILTER(PROMPT('There are lemons in the kitchen: {0}', DESCRIPTION));

---

## Extracting Insights from Documents Using AISQL 📄🤖
In this section, we will demonstrate how to apply Snowflake Cortex AISQL functions to analyze and extract structured information from unstructured document content—specifically, resumes.

Using AISQL, we will:

- Parse the content of uploaded resumes
- Identify and extract key information such as:
    - ✅ Technical skills
    - 🎓 Certifications
    - 💼 Experience with specific technologies

In [ ]:
-- Explore the resumes path in our local stage
SELECT *
FROM DIRECTORY('@FRANCISCO_STAGE')
WHERE STARTSWITH(RELATIVE_PATH,'resumes');

In [ ]:
-- Extract information using AISQL functions and save the results to a temporary table
CREATE OR REPLACE TEMPORARY TABLE RESUME_DATA AS

SELECT
    -- Resume
    RELATIVE_PATH,

    -- Ask specific questions
    AI_EXTRACT( file => TO_FILE('@FRANCISCO_STAGE', RELATIVE_PATH),
     responseFormat => [
     ['skills','Extract the software engineering technical skills from this resume. Reply with an array with unique identified technical skills.'],
     ['certifications','Extract the technical certifications from this resume. Return an array of certifications.'],
     ['is_cloud_engineer','Based in this resume content, condering job experience and certifications, is this person a Cloud Engineer? Reply only with " "true" or "false"'],
     ['is_data_engineer','Based in this resume content, condering job experience and certifications, is this person a Data Engineer? Reply only with " "true" or "false"'],
     ['is_java_developer','Based in this resume content, condering job experience and certifications, Does this person have Java knowledge? Reply only with " "true" or "false"'],
     ['is_snowflake_dev','Based in this resume content, condering job experience and certifications, Does this person have Snowflake knowledge? Reply only with " "true" or "false"'],
     ['snowflake_experience','What is the level of experience in Snowflake based in this resume content? Consider job experience, certifications and seniority. Reply with only one of these options: Beginner, Basic, Intermediate, Advanced or Expert.'],
     ['java_experience','What is the level of experience in Java based in this resume content? Consider job experience, certifications and seniority. Reply with only one of these options: Beginner, Basic, Intermediate, Advanced or Expert.']
     ]
    ) AS DATA_EXTRACT,

    -- Parse the entire document content
    AI_PARSE_DOCUMENT (TO_FILE('@FRANCISCO_STAGE', RELATIVE_PATH), {'mode': 'LAYOUT' , 'page_split': false}) AS RESUME_CONTENT

FROM DIRECTORY('@FRANCISCO_STAGE')
WHERE STARTSWITH(RELATIVE_PATH,'resumes');

In [ ]:
-- List the resumes and the extracted information
SELECT * FROM RESUME_DATA;

In [ ]:
-- Display results in a user-friendly format
SELECT
    RELATIVE_PATH,
    DATA_EXTRACT:response:is_cloud_engineer::boolean AS IS_CLOUD_ENG,
    DATA_EXTRACT:response:is_data_engineer::boolean AS IS_DATA_ENG,
    DATA_EXTRACT:response:is_java_developer::boolean AS IS_JAVA_DEV,
    -- DATA_EXTRACT:response:is_databricks_dev::boolean AS IS_DATABRICKS_DEV,
    DATA_EXTRACT:response:is_snowflake_dev::boolean AS IS_SNOWFLAKE_DEV,
    DATA_EXTRACT:response:snowflake_experience::string AS SNOWFLAKE_SKILL_LEVEL,
    DATA_EXTRACT:response:java_experience::string AS JAVA_SKILL_LEVEL,
    TO_ARRAY(DATA_EXTRACT:response:certifications) AS CERTIFICATIONS
FROM RESUME_DATA;

In [ ]:
-- Find people with experience
SELECT
    RELATIVE_PATH,
FROM RESUME_DATA
WHERE AI_FILTER(PROMPT('This person has experience in the Financial or Health Services sectors, {0}', RESUME_CONTENT:content)) = TRUE;

## Thank You! 🙏
Thank you for exploring this Snowflake Cortex AISQL demonstration notebook. I hope it provided valuable insights into how generative AI can be used to extract meaningful information from unstructured data such as images and documents—all within the Snowflake ecosystem.

✅ Key Takeaways

- Cortex AISQL functions enable powerful, scalable AI-driven analysis directly in SQL
- You can extract structured insights from images, documents, and text with minimal setup
- Integration with Snowflake stages and tables makes it easy to operationalize AI workflows

👤 Author

Francisco Leyva

🔗 [LinkedIn Profile](https://www.linkedin.com/in/franciscoleyva/)